In [1]:
import pandas as pd
import numpy as np
import requests
from io import BytesIO
import os
import yadisk
import gdown
import json

import config
from parse_functions import *
from yandex_disk_func import *
from google_connector import *

# ссылка на основной файл гугл докс (на каждом лите отдельный клиент со ссылками на файлы с данными)
base_link = config.base_link
yandex_token = config.yandex_token

# указываем путь к основной папке, в которой храняться папки с флайтами
main_folder = config.main_folder # путь к папке в формате /tmp
public_key = config.public_key # обычная ссылка на доступ к папке  из личного кабинета

# забираем список папок в нужной нам директории
res = get_yandex_disk_folders(public_key)
yandex_folders = res.json() # парсим ответ

200


In [2]:
# Включаем отображение всех колонок
pd.set_option('display.max_columns', None)
# Задаем ширину столбцов по контенту
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)
pd.options.mode.chained_assignment = None

In [3]:
# забираем данные из основной управленческой таблицы
content = get_data_from_sheet(base_link)
sheet_names = pd.ExcelFile(content)

In [4]:
sheet_names.sheet_names

['Калининград МореАвто(ЮтоКарс)',
 'Екатеринбург АтлантикПРО',
 'АвтоБелогорье',
 'DoubleTint']

In [5]:
# если появился новый клиент
# мы проверяем, что этого названия НЕТ среди папок на Яндекс Диске
# созлаем папку Клиента
# внутри нее создаем папку для файлов дашбордов
folders_list = get_folder_name(yandex_folders['_embedded']['items'])
for name in sheet_names.sheet_names:
    if name not in folders_list:
        print(name)
        dashboard_folder = '/' + name + '/Файлы_дашбордов'
        create_yandex_disk_folder(main_folder, name, yandex_token)
        create_yandex_disk_folder(main_folder, dashboard_folder, yandex_token)

Калининград МореАвто(ЮтоКарс)
Екатеринбург АтлантикПРО
АвтоБелогорье
DoubleTint


In [6]:
# обновляем список папок на Яндекс диске
res = get_yandex_disk_folders(public_key)
yandex_folders = res.json() # парсим ответ

# проходим через цикл по всем листам в управленческой таблице
# вызываем функцию и забираем названия папок на яндекс диске
folders_list = get_folder_name(yandex_folders['_embedded']['items'])

for name in sheet_names.sheet_names:
    print(name)
    
    cols_range = 'A:B' # задаем диапазон полей, которые нам нужны
    df = pd.read_excel(content, sheet_name=name, usecols=cols_range, names=['report_name', 'report_link'])
    # нормализуем назания отчетов - приводим в нижний регистр, обрезаем по краям, избаляемся от технических символов(перенос строки и тд)
    df['report_name'] = df['report_name'].str.lower().str.strip().str.replace('\n', ' ')
    # создаем отдельный список с названиями отчетов и проодим по нему через цикл
    reports_list = list(df['report_name'])
    # запускаем блок парсинга отчетов
    
    
    for report in reports_list:
        if report == 'калининград ап':
            print(report)
            parse_info = df[df['report_name']==report].reset_index(drop=True)
            base_link = parse_info['report_link'][0]
            
            get_data_to_yandex_disk(main_folder, base_link, name, report, yandex_token)

200
Калининград МореАвто(ЮтоКарс)
калининград ап
Файл успешно загружен
Екатеринбург АтлантикПРО
АвтоБелогорье
DoubleTint


In [ ]:
import io
import os

In [ ]:
path = r'C:\Users\o.bogomolov\Desktop\Jupyter_notebook\34_03_download_files_from_google_to_yandex\test'
file_name = 'test.xlsx'

final_path = os.path.join(path, file_name)

In [ ]:
import requests
response = requests.get('https://docs.google.com/spreadsheets/d/16z9hgAKPyWmZ0xGyHsUlq4knK-idq50WJRgOT2R5fts/export?format=xlsx')

In [ ]:
with open(final_path, 'wb') as file:
        file.write(response.content)

In [ ]:
import gdown

In [ ]:
path = r'C:\Users\o.bogomolov\Desktop\Jupyter_notebook\34_03_download_files_from_google_to_yandex\test'

In [ ]:
folder_path = 'https://drive.google.com/drive/folders/1A6RHtXKIMneWUkD-Fs2oLrfbVwDk07Er?usp=sharing'

In [ ]:
gdown.download_folder(folder_path, output=path, quiet=True, use_cookies=False)